In [1]:
print('hello')

hello


In [2]:
from testing_arcgis import *

In [3]:
cif = CIFTool_AGOL( 'https://ky-cancer.maps.arcgis.com', 'zch250@uky.edu', 'aBYdELneCf6cnT2')
# cif("Dartmouth_catchment_data\dataset.pickle")


Successfully logged in as: zch250@uky.edu
folder name is Mon Nov 14 14:20:03 2022


In [4]:
cif.read_pickle("Dartmouth_catchment_data\dataset.pickle")

In [5]:
cif.data_dictionary.keys()

dict_keys(['Screening & Risk Factors (County)', 'Screening & Risk Factors (Tract)', 'Cancer Incidence - Age Adj.', 'Cancer Mortality - Age Adj.', 'Economic Indicators (County)', 'Economic INdicators (Tract)', 'Housing & Transportation (County)', 'Housing & Transportation (Tract)', 'Sociodemographics (County)', 'Sociodemographics (Tract)', 'Environment (County)', 'Environment (Tract)', 'Broadband Speed', 'Facilities and Providers'])

In [6]:
cif.genPointFL()

  0%|          | 0/4 [00:00<?, ?it/s]

Waiting to Upload Facilities and Providers : GI Providers layer to AGOL:   0%|          | 0/30 [00:00<?, ?it/s…

Uploading Point Feature Layer : GI Providers:   0%|          | 0/30 [00:00<?, ?it/s]

Updating layer name to 'GI Providers':   0%|          | 0/30 [00:00<?, ?it/s]

Waiting to Upload Facilities and Providers : FQHCs / Other HPSA layer to AGOL:   0%|          | 0/30 [00:00<?,…

...

Uploading Point Feature Layer : Lung Cancer Screening:   0%|          | 0/30 [00:00<?, ?it/s]

Updating layer name to 'Lung Cancer Screening':   0%|          | 0/30 [00:00<?, ?it/s]

Waiting to Upload Facilities and Providers : Mammography layer to AGOL:   0%|          | 0/30 [00:00<?, ?it/s]

Uploading Point Feature Layer : Mammography:   0%|          | 0/30 [00:00<?, ?it/s]

Updating layer name to 'Mammography':   0%|          | 0/30 [00:00<?, ?it/s]

In [7]:
names = list(cif.countyData.keys())
dfs    = list(cif.countyData.values())
print("Generating Web Feature Layers for County Level Data")
for i in trange(1):
    df = dfs[i]
    name = names[i]
    cif.genAreaSDF4FL_new(df, name, 'county', keep_NA = True)

Generating Web Feature Layers for County Level Data


  0%|          | 0/1 [00:00<?, ?it/s]

Waiting to Upload Screening & Risk Factors (County) layer to AGOL:   0%|          | 0/30 [00:00<?, ?it/s]

Updating layer name to 'Screening & Risk Factors (County)':   0%|          | 0/30 [00:00<?, ?it/s]

In [8]:
cif.shareFL()


In [21]:
contentManager = cif._contentManager
area_layers = {key:value for key, value in cif.groupLayersCP.items() if not bool(re.match("Facilities.+", key))}
area_layers = {key:value for key, value in area_layers.items() if not bool(re.match(".+(Tract)", key))}

In [22]:
area_layers

{'Screening & Risk Factors (County)': {'id': '49c9dcc4d5cb4d048ad73cdf5bccb758',
  'fields': ['Obese__BMI',
   'Cancer_Pre',
   'Currently_',
   'Met_Breast',
   'Had_Pap_Te',
   'Met_Colore'],
  'alias': ['Obese (BMI over 30)',
   'Cancer Prevalence',
   'Currently Smoke (Adult)',
   'Met Breast Screening Recommendations',
   'Had Pap Test in Last 3 Years, Age 21-64',
   'Met Colorectal Screening Recommendations']}}

In [23]:
fl_names = list(area_layers.keys())
fl_dicts = list(area_layers.values())
fl_length = len(fl_names)
for j in trange(fl_length):
    flname = fl_names[j]
    fl_dict = fl_dicts[j]
    webmap_ids = {}
    GL_id = fl_dict['id']
    GL = contentManager.search(GL_id)[0]
#         for flname, fl_dict in area_layers.items():
    # first edit alias of each field
    if j == 0:
        extent = dict(zip(['xmin','ymin','xmax','ymax'], GL.extent[0]+ GL.extent[1]))
        extent['spatialReference'] = GL.spatialReference
    lyr = GL.layers[0]
    properties = lyr.properties
    original_colname = fl_dict['alias']
    AGOLNames = fl_dict['fields']
    sdf_colname = [x.name for x in properties.fields]
    colname_dict = dict(zip(original_colname, AGOLNames))

for colname in fl_dict['alias']:
    AGOLName = colname_dict[colname]
    idx = sdf_colname.index(AGOLName)
#                 idx, AGOLName = self.getAGOLFieldName(lyr, colname)
    properties['fields'][idx]['alias'] = colname
lyr.properties = properties
time.sleep(.25)
# next, define a renderer for each and create webmaps

  0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
colname, title = fl_dict['fields'][0], fl_dict['alias'][0]

In [25]:
colname, title

('Obese__BMI', 'Obese (BMI over 30)')

In [26]:
renderer = cif.renderer_definition(lyr, colname)
new_renderer = cif.add_class_details(renderer)
wm = WebMap()
wm.add_layer(lyr, {'title': f'{flname} : {title}',
                   'renderer': new_renderer})
# wm.add_layer(item, {'title': f'{title} : {key}',
#                    'renderer' : new_renderer})
cif.wait_AGOL(.5, desc = f'Adding the layer "{flname}:{title}" to the map')

Adding the layer "Screening & Risk Factors (County):Obese (BMI over 30)" to the map:   0%|          | 0/30 [00…

In [27]:
wm = cif.manage_popups(wm, level = 'county', title = f"{flname} : {title}", labels_to_select = original_colname)


In [29]:
cif.wait_AGOL(.5, desc = f"Managing Popup")
wm = cif.addPointLayers(wm, start_index = 1)
webmap_item_properties = {'title': f'{flname} : {title}',
                         'snippet':f'{flname} : {title}',
                         'tags' : ['Python','Automated'],
                          'extent': extent
                         }
webmap = wm.save(webmap_item_properties, folder = cif.AGOL_folder) # testing folder is temporary set-up
cif.wait_AGOL(.5, desc = f'Creating a WebMap "{flname}:{title}"')
#                         searchedWebMap = contentManager.search(f'title:"{flname} : {title}"', item_type = 'Web Map')
#                         webmap_s = searchedWebMap[0]
#                         webmap_s.share(everyone = True)
#                         self.wait_AGOL(.5, desc = f"Chaning sharing option")
webmap_id = webmap.id
webmap_ids[title] = webmap_id
flag = False
cif.wait_AGOL(1, desc = f"Complete: {flname}-{title}")


Managing Popup:   0%|          | 0/30 [00:00<?, ?it/s]

Creating a WebMap "Screening & Risk Factors (County):Obese (BMI over 30)":   0%|          | 0/30 [00:00<?, ?it…

Complete: Screening & Risk Factors (County)-Obese (BMI over 30):   0%|          | 0/30 [00:00<?, ?it/s]

In [30]:
wm

AttributeError: 'super' object has no attribute '_ipython_display_'

WebMap()

In [23]:
labels_to_select = original_colname
lyr_1 = wm.layers[0]
if title is None:
    lyr_1.popupInfo.title = .popupInfo.title.replace("_", " ").title()
elif isinstance(title, str):
    lyr.popupInfo.title = title
else:
    raise ValueError("title must be a string")
if labels_to_select:
    main_fields = [f.label for f in lyr.popupInfo.fieldInfos if f.label in labels_to_select]
else:
    main_fields = [f.label for f in lyr.popupInfo.fieldInfos if f.fieldName != f.label]
try:
    main_fields.remove("Fips")
except:
    pass



In [25]:
for field in lyr.popupInfo.fieldInfos:
    if field.label.islower():
        field.label = field.label.title()
    if level.lower() == 'county':
        labels_to_include  = ['County','State'] + main_fields
    elif level.lower() == 'tract':
        labels_to_include = ['Tract','County','State'] + main_fields
    else:
        raise ValueError('level must be either "county" or "tract"')

    if field.label in labels_to_include:
        field.visible = True
    else:
        field.visible = False


In [26]:
wm

AttributeError: 'super' object has no attribute '_ipython_display_'

WebMap()

In [ ]:
    @staticmethod
    def manage_popups(wm, level = 'county', title  = None, labels_to_select = None):
        for lyr in wm.layers:
            if title is None:
                lyr.popupInfo.title = lyr.popupInfo.title.replace("_", " ").title()
            elif isinstance(title, str):
                lyr.popupInfo.title = title
            else:
                raise ValueError("title must be a string")
            if labels_to_select:
                main_fields = [f.label for f in lyr.popupInfo.fieldInfos if f.label in labels_to_select]
            else:
                main_fields = [f.label for f in lyr.popupInfo.fieldInfos if f.fieldName != f.label]
            try:
                main_fields.remove("Fips")
            except:
                pass
            for field in lyr.popupInfo.fieldInfos:
                if field.label.islower():
                    field.label = field.label.title()
                if level.lower() == 'county':
                    labels_to_include  = ['County','State'] + main_fields
                elif level.lower() == 'tract':
                    labels_to_include = ['Tract','County','State'] + main_fields
                else:
                    raise ValueError('level must be either "county" or "tract"')

                if field.label in labels_to_include:
                    field.visible = True
                else:
                    field.visible = False
        return wm


In [ ]:

    def genWebMapFromGroupLayers(self, level= 'county'):
        if self._contentManager:
            contentManager = self._contentManager
        else:
            contentManager = self.gis
        area_layers = {key:value for key, value in self.groupLayersCP.items() if not bool(re.match("Facilities.+", key))}
        if level.lower() == 'county':
            area_layers = {key:value for key, value in area_layers.items() if not bool(re.match(".+(Tract)", key))}
        elif level.lower() == 'tract':
            area_layers = {key:value for key, value in area_layers.items() if bool(re.match(".+(Tract)", key))}
        else:
            raise ValueError("level can ben either 'county' or 'tract'")
        fl_names = list(area_layers.keys())
        fl_dicts = list(area_layers.values())
        fl_length = len(fl_names)
        for j in trange(fl_length):
            flname = fl_names[j]
            fl_dict = fl_dicts[j]
            webmap_ids = {}
            GL_id = fl_dict['id']
            GL = contentManager.search(GL_id)[0]
#         for flname, fl_dict in area_layers.items():
            # first edit alias of each field
            if j == 0:
                extent = dict(zip(['xmin','ymin','xmax','ymax'], GL.extent[0]+ GL.extent[1]))
                extent['spatialReference'] = GL.spatialReference
            lyr = GL.layers[0]
            properties = lyr.properties
            original_colname = fl_dict['alias']
            AGOLNames = fl_dict['fields']
            sdf_colname = [x.name for x in properties.fields]
            colname_dict = dict(zip(original_colname, AGOLNames))

            for colname in fl_dict['alias']:
                AGOLName = colname_dict[colname]
                idx = sdf_colname.index(AGOLName)
#                 idx, AGOLName = self.getAGOLFieldName(lyr, colname)
                properties['fields'][idx]['alias'] = colname
            lyr.properties = properties
            time.sleep(.25)
            # next, define a renderer for each and create webmaps
            for colname, title in zip(fl_dict['fields'], fl_dict['alias']):
                renderer = self.renderer_definition(lyr, colname)
                new_renderer = self.add_class_details(renderer)
                flag = True
                while flag:
                    try:
                        wm = WebMap()
                        wm.add_layer(lyr, {'title': f'{flname} : {title}',
                                           'renderer': new_renderer})
                        # wm.add_layer(item, {'title': f'{title} : {key}',
                        #                    'renderer' : new_renderer})
                        self.wait_AGOL(.5, desc = f'Adding the layer "{flname}:{title}" to the map')
                        wm = self.manage_popups(wm, level = level, title = f"{flname} : {title}", labels_to_select = original_colname)
                        self.wait_AGOL(.5, desc = f"Managing Popup")
                        wm = self.addPointLayers(wm, start_index = 1)
                        webmap_item_properties = {'title': f'{flname} : {title}',
                                                 'snippet':f'{flname} : {title}',
                                                 'tags' : ['Python','Automated'],
                                                  'extent': extent
                                                 }
                        webmap = wm.save(webmap_item_properties, folder = self.AGOL_folder) # testing folder is temporary set-up
                        self.wait_AGOL(.5, desc = f'Creating a WebMap "{flname}:{title}"')
#                         searchedWebMap = contentManager.search(f'title:"{flname} : {title}"', item_type = 'Web Map')
#                         webmap_s = searchedWebMap[0]
#                         webmap_s.share(everyone = True)
#                         self.wait_AGOL(.5, desc = f"Chaning sharing option")
                        webmap_id = webmap.id
                        webmap_ids[title] = webmap_id
                        flag = False
                        self.wait_AGOL(1, desc = f"Complete: {flname}-{title}")
                    except:
                        time.sleep(2)
                        self.error_count += 1
                        if self.error_count >= 5:
                            contentManager = self.gis
                            self.error_count = 0
            self.webmaps[flname] = webmap_ids
            del self.groupLayersCP[flname]
        self.shareWebMaps(level = level)
        return self.webmaps


In [11]:
    def genWebMapFromGroupLayers(self, level= 'county'):
        if self._contentManager:
            contentManager = self._contentManager
        else:
            contentManager = self.gis
        area_layers = {key:value for key, value in self.groupLayersCP.items() if not bool(re.match("Facilities.+", key))}
        if level.lower() == 'county':
            area_layers = {key:value for key, value in area_layers.items() if not bool(re.match(".+(Tract)", key))}
        elif level.lower() == 'tract':
            area_layers = {key:value for key, value in area_layers.items() if bool(re.match(".+(Tract)", key))}
        else:
            raise ValueError("level can ben either 'county' or 'tract'")
        fl_names = list(area_layers.keys())
        fl_dicts = list(area_layers.values())
        fl_length = len(fl_names)
        for j in trange(fl_length):
            flname = fl_names[j]
            fl_dict = fl_dicts[j]
            webmap_ids = {}
            GL_id = fl_dict['id']
            GL = contentManager.search(GL_id)[0]
#         for flname, fl_dict in area_layers.items():
            # first edit alias of each field
            if j == 0:
                extent = dict(zip(['xmin','ymin','xmax','ymax'], GL.extent[0]+ GL.extent[1]))
                extent['spatialReference'] = GL.spatialReference
            lyr = GL.layers[0]
            properties = lyr.properties
            original_colname = fl_dict['alias']
            AGOLNames = fl_dict['fields']
            sdf_colname = [x.name for x in properties.fields]
            colname_dict = dict(zip(original_colname, AGOLNames))

            for colname in fl_dict['alias']:
                AGOLName = colname_dict[colname]
                idx = sdf_colname.index(AGOLName)
#                 idx, AGOLName = self.getAGOLFieldName(lyr, colname)
                properties['fields'][idx]['alias'] = colname
            lyr.properties = properties
            time.sleep(.25)
            # next, define a renderer for each and create webmaps
            for colname, title in zip(fl_dict['fields'], fl_dict['alias']):
                renderer = self.renderer_definition(lyr, colname)
                new_renderer = self.add_class_details(renderer)
                flag = True
                while flag:
                    try:
                        wm = WebMap()
                        wm.add_layer(lyr, {'title': f'{flname} : {title}',
                                           'renderer': new_renderer})
                        # wm.add_layer(item, {'title': f'{title} : {key}',
                        #                    'renderer' : new_renderer})
                        self.wait_AGOL(.5, desc = f'Adding the layer "{flname}:{title}" to the map')
                        wm = self.manage_popups(wm, level = level, title = f"{flname} : {title}")
                        self.wait_AGOL(.5, desc = f"Managing Popup")
                        wm = self.addPointLayers(wm, start_index = 1)
                        webmap_item_properties = {'title': f'{flname} : {title}',
                                                 'snippet':f'{flname} : {title}',
                                                 'tags' : ['Python','Automated'],
                                                  'extent': extent
                                                 }
                        webmap = wm.save(webmap_item_properties, folder = self.AGOL_folder) # testing folder is temporary set-up
                        self.wait_AGOL(.5, desc = f'Creating a WebMap "{flname}:{title}"')
#                         searchedWebMap = contentManager.search(f'title:"{flname} : {title}"', item_type = 'Web Map')
#                         webmap_s = searchedWebMap[0]
#                         webmap_s.share(everyone = True)
#                         self.wait_AGOL(.5, desc = f"Chaning sharing option")
                        webmap_id = webmap.id
                        webmap_ids[title] = webmap_id
                        flag = False
                        self.wait_AGOL(1, desc = f"Complete: {flname}-{title}")
                    except:
                        time.sleep(2)
                        self.error_count += 1
                        if self.error_count >= 5:
                            contentManager = self.gis
                            self.error_count = 0
            self.webmaps[flname] = webmap_ids
            del self.groupLayersCP[flname]
        self.shareWebMaps(level = level)
        return self.webmaps


  0%|          | 0/1 [00:00<?, ?it/s]

Adding the layer "Screening & Risk Factors (County):Obese (BMI over 30)" to the map:   0%|          | 0/30 [00…

Managing Popup:   0%|          | 0/30 [00:00<?, ?it/s]

Creating a WebMap "Screening & Risk Factors (County):Obese (BMI over 30)":   0%|          | 0/30 [00:00<?, ?it…

Complete: Screening & Risk Factors (County)-Obese (BMI over 30):   0%|          | 0/30 [00:00<?, ?it/s]

KeyboardInterrupt: 